In [25]:
import socket
import struct
import time

### Connection
# Connect to a Betaflight drone via TCP
host = '192.168.1.2'  # IP address of the drone
port = 5761  # port typically used for TCP-based connections with Betaflight

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((host, port))

In [34]:
### MSP commands
# out messages usually start with 1 (1**)
MSP_STATUS = 101
MSP_RAW_IMU = 102
# in messages usually start with 2 (2**)
MSP_SET_MOTOR = 214

### Main functions
def construct_msp_message(command: int, data: list = []) -> bytearray:
    """Creates expected formatting.

    Command frame format:
    Header: `$M<`
    Body: length of `data` + `command` + `data` values
    Tail: checksum

    Args:
        command (int): MSP command
        data (list, optional): list of params (length is varying). Defaults to [].

    Returns:
        bytearray: command frame
    """
    message = bytearray()
    # header
    message.append(ord('$'))
    message.append(ord('M'))
    message.append(ord('<'))
    # body
    message.append(len(data))
    message.append(command)
    checksum = len(data) ^ command
    for byte in data:
        message.append(byte)
        checksum ^= byte
    # tail
    message.append(checksum)
    return message

def send_msp_message(command: int, data: list = []):
    """Send prepared command via serial

    Args:
        command (int): MSP command
        data (list, optional): list of params. Defaults to [].
    """
    message = construct_msp_message(command, data)
    sock.sendall(message)

def read_msp_response() -> tuple:
    """Read data

    Returns:
        tuple: command code and its values
    """
    header = sock.recv(3)  # read header
    if header != b'$M>':
        return None
    # read body
    data_length = sock.recv(1)[0]
    code = sock.recv(1)[0]
    data = sock.recv(data_length)
    # read tail
    checksum = sock.recv(1)[0]
    # check if the command is of the expected length
    calculated_checksum = data_length ^ code
    for byte in data:
        calculated_checksum ^= byte
    if calculated_checksum != checksum:
        return None

    return (code, data)

def get_raw_imu() -> tuple:
    """Read raw IMU data and decode it as signed 16-bit integers.

    Returns:
        tuple | None: tuple - processed data, None - received data doesn't correspond to sent command
    """
    send_msp_message(MSP_RAW_IMU)  # send request
    time.sleep(0.009)  # wait for reply
    response = read_msp_response()  # get reply

    if response and response[0] == MSP_RAW_IMU:
        # Decode as signed 16-bit integers
        imu = struct.unpack('<9h', response[1])  # AccX, AccY, AccZ, GyroX, GyroY, GyroZ
        #imu = struct.unpack(f'<{len(response[1]) // 2}H', response[1])  # decode
        return imu[:3] # AccX, AccY, AccZ
    return None

In [45]:
import numpy as np

class PoseEstimator:
    def __init__(self, dt, process_noise, measurement_noise):
        # Time step
        self.dt = dt

        # State vector [position_x, position_y, position_z, velocity_x, velocity_y, velocity_z]
        self.x = np.zeros((6, 1))

        # State transition matrix
        self.A = np.eye(6)
        for i in range(3):
            self.A[i, i + 3] = dt  # Position update from velocity

        # Control matrix for accelerometer input (assumes constant acceleration within dt)
        self.B = np.zeros((6, 3))
        for i in range(3):
            self.B[i + 3, i] = dt

        # Process noise covariance (estimated noise in the system)
        self.Q = process_noise * np.eye(6)

        # Measurement noise covariance (estimated sensor noise)
        self.R = measurement_noise * np.eye(3)

        # Measurement matrix to relate state to position measurements
        self.H = np.zeros((3, 6))
        for i in range(3):
            self.H[i, i] = 1

        # Initial covariance matrix
        self.P = np.eye(6)

    def predict(self, u):
        # Predict the state
        self.x = np.dot(self.A, self.x) + np.dot(self.B, u.reshape(3, 1))  # Reshape u to (3, 1)
        
        # Predict the covariance
        self.P = np.dot(self.A, np.dot(self.P, self.A.T)) + self.Q

    def update(self, z):
        # Measurement residual
        y = z - np.dot(self.H, self.x)
        
        # Residual covariance
        S = np.dot(self.H, np.dot(self.P, self.H.T)) + self.R
        
        # Kalman gain
        K = np.dot(self.P, np.dot(self.H.T, np.linalg.inv(S)))
        
        # Update the state
        self.x = self.x + np.dot(K, y)
        
        # Update the covariance
        I = np.eye(self.P.shape[0])
        self.P = np.dot(I - np.dot(K, self.H), self.P)

    def get_position(self):
        # Return only the current position estimate (x, y, z)
        return self.x[0:3].flatten()

# Parameters
dt = 0.01  # Time step
process_noise = 2.56
measurement_noise = 1.0

# Initialize Kalman filter
pose_estimator = PoseEstimator(dt, process_noise, measurement_noise)

# Simulation loop
while True:
    # Replace `get_raw_imu()` with actual IMU data retrieval
    acceleration = np.array(get_raw_imu())  # Example: get acceleration data [ax, ay, az]
    
    # Ensure the acceleration is a 3-element array (reshape if needed)
    if acceleration.shape[0] != 3:
        raise ValueError("Acceleration input must be a 3-element vector.")

    pose_estimator.predict(u=acceleration)  # Predict with acceleration as control input
    measured_position = np.zeros((3, 1))  # Assuming no external measurement for this example
    pose_estimator.update(z=measured_position)  # Update with "no measurement" to simulate pure integration
    
    # Get and print only the current x, y, z position estimate
    position = pose_estimator.get_position()
    print(f"Position (x, y, z): {position[0]:.2f}, {position[1]:.2f}, {position[2]:.2f}")


Position (x, y, z): 0.00, 0.00, 0.00
Position (x, y, z): -0.03, 0.04, 0.02
Position (x, y, z): -0.06, 0.08, 0.04
Position (x, y, z): -0.06, 0.09, 0.09
Position (x, y, z): -0.07, 0.09, 0.15
Position (x, y, z): -0.06, 0.08, 0.21
Position (x, y, z): -0.06, 0.08, 0.27
Position (x, y, z): -0.06, 0.07, 0.34
Position (x, y, z): -0.06, 0.07, 0.40
Position (x, y, z): -0.06, 0.06, 0.46
Position (x, y, z): -0.06, 0.06, 0.52
Position (x, y, z): -0.06, 0.06, 0.58
Position (x, y, z): -0.06, 0.06, 0.65
Position (x, y, z): -0.06, 0.05, 0.71
Position (x, y, z): -0.06, 0.05, 0.77
Position (x, y, z): -0.06, 0.05, 0.83
Position (x, y, z): -0.06, 0.04, 0.89
Position (x, y, z): -0.06, 0.04, 0.95
Position (x, y, z): -0.06, 0.04, 1.01
Position (x, y, z): -0.06, 0.04, 1.07
Position (x, y, z): -0.06, 0.04, 1.13
Position (x, y, z): -0.06, 0.04, 1.19
Position (x, y, z): -0.06, 0.04, 1.25
Position (x, y, z): -0.06, 0.03, 1.31
Position (x, y, z): -0.06, 0.03, 1.37
Position (x, y, z): -0.06, 0.03, 1.43
Position (x, 

KeyboardInterrupt: 

In [31]:
get_raw_imu()

(23, 2042, 309, 1, 4, 1)